В ходе работы будут рассмотрены основные алгоритмы скрытых Марковсих моделей. 
Скрытая Марковская модель или кратко HMM формально может быть описана следующим образом.

* Множество дискретных состояний $S = (S_1,S_2,... S_M)$
* Соостояние в момент времени $t: q_t$ 
* Множество возможных сигналов $V = (V_1,V_2,... V_K)$
* Матрица переходов $A$ из состояния  в состояние на мн-ве $S, A_{ij} = P(q_{t+1} = S_j | q_t = S_i)$
* Матрица $B$ , которая содержит вероятности выдать сигнал в определенном состоянии 
    $B_{ij} = b_i(j) = P ( v_j | q_t = S_i )$
* Наблюдаемая последовательность сигналов $O=(O_1,O_2,...,O_L)$
* Вектор начального состояния $\pi , \pi_i = P(q_1 = S_i)$ 
* HMM Model  $\lambda = 	(A,B, \pi)$

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
M = int(input('Введите количество состояний:'))

Введите количество состояний:6


In [3]:
K = int(input('Введите количество возможных сигналов:'))

Введите количество возможных сигналов:8


In [4]:
L = int(input('Введите длину наблюдаемой последовательности:'))

Введите длину наблюдаемой последовательности:10


#### 1). Генерация параметров модели
Сгенерируем матрицу переходов, матрицу эмиссий и вектор начального состояния.

In [5]:
## возвращает матрицу размера NxM с именами столбцов prefix1,prefix2
def generate_random_probability_matrix(N, M, prefix1, prefix2):
    matrix = pd.DataFrame(np.random.randint(0,10,size=(N, M))).add_prefix(prefix2).T.add_prefix(prefix1).T
    matrix = matrix.div(matrix.sum(axis=1),axis=0)
    return matrix

A = generate_random_probability_matrix(M,M,'p','p')
B = generate_random_probability_matrix(M,K,'p','e')
pi = generate_random_probability_matrix(M,1,'p','')

In [6]:
A = generate_random_probability_matrix(M,M,'p','p')
B = generate_random_probability_matrix(M,K,'p','e')
pi = generate_random_probability_matrix(1,M,'','p')

И запишем сгенерированные значения в файлы A.csv, B.csv, pi.csv соответственно

In [7]:
A.to_csv('A.csv',sep='\t')
B.to_csv('B.csv',sep='\t')
pi.to_csv('pi.csv',sep='\t')

#### 2). Прочитаем параметры модели из файлов 

In [8]:
class HMM(object):
    def __init__(self, initialProb, transMatrix, emissionMatrix, M, K):
        self.M = M
        self.K = K
        self.initialProb = initialProb
        self.transMatrix = transMatrix
        self.emissionMatrix = emissionMatrix
        self.obs_set = ['p'+ str(i) for i in range(M)]
        self.emm_set = ['e'+ str(i) for i in range(K)]

In [9]:
A = pd.read_csv('A.csv',sep='\t',index_col=0)
B = pd.read_csv('B.csv',sep='\t',index_col=0)
pi = pd.read_csv('pi.csv',sep='\t',index_col=0)

hmm = HMM(pi,A,B,M,K)

#### 3). Сгенерируем наблюдаемую последовательность сигналов и последовательность состояний

In [10]:
x = np.random.randint(0,K,size=L)
O = ['e'+str(x[i]) for i in range(L)]
O

['e3', 'e2', 'e4', 'e1', 'e1', 'e1', 'e7', 'e2', 'e0', 'e3']

#### 4). Алгоритм Витерби.
Используем алгоритм Витерби для нахождения наиболее вероятной последовательности состояний, соответствующей последовательности наблюдаемых сигналов. Учитываются параметры модели HMM.

In [11]:
def viterbiDecode(hmm, obs):
    L = len(obs)
    delta = pd.DataFrame(np.zeros((hmm.M, L))).add_prefix('t').T.add_prefix('p').T
    backpt = pd.DataFrame([['' for i in range(L)] for j in range(hmm.M)]).add_prefix('t').T.add_prefix('p').T

    #init
    for i in hmm.obs_set:
        delta.loc[i]['t0'] = hmm.initialProb[i]*hmm.emissionMatrix.loc[i][obs[0]]
        backpt.loc[i]['t0'] = 'p0'
        
    #recursion
    for t in range(1, L):
        for j in hmm.obs_set:
            v1 = []
            v2 = []
            for i in hmm.obs_set:
                v1.append(delta.loc[i][t-1]*hmm.transMatrix.loc[i][j]*hmm.emissionMatrix.loc[j][obs[t]])
                v2.append(delta.loc[i][t-1]*hmm.transMatrix.loc[i][j])
            v1 = np.array(v1)
            v2 = np.array(v2)
            delta.loc[j][t] = v1.max(0)
            backpt.loc[j][t] = 'p'+str(v2.argmax(0))
        
    # termination
    ind_opt = ['' for i in range(L)]
    p_opt = delta['t'+str(L-1)].max()
    ind_opt[L-1] = (delta['t'+str(L-1)]).idxmax()

    for t in range(L-2,-1, -1):
        ind_opt[t] = backpt.loc[ind_opt[t+1]]['t'+str(t+1)]
    
    return ind_opt

In [12]:
print(viterbiDecode(hmm, O))

['p0', 'p2', 'p1', 'p4', 'p4', 'p4', 'p0', 'p2', 'p1', 'p2']


#### 5). Алгоритм forward

In [13]:
def forward(hmm, obs):
    L = len(obs)
    fwd = pd.DataFrame(np.zeros((hmm.M,L)),index=hmm.transMatrix.index)

    for i in range(L):
        for s in hmm.obs_set:
            if i == 0:
                prev_f_sum = hmm.initialProb[s]
            else:
                prev_f_sum = 0
                for k_i,k in enumerate(hmm.obs_set):
                    prev_f_sum += fwd[i-1][k_i]*hmm.transMatrix.loc[k][s]
            fwd[i][s] =  prev_f_sum*hmm.emissionMatrix.loc[s][obs[i]]
    return fwd

def probability_fwd(fwd, obs):
    L = len(obs)
    return fwd[L-1].sum()

In [14]:
obs = O
fwd = forward(hmm, obs)
fwd

,0,1,2,3,4,5,6,7,8,9
p0,0.028361,0.000467,0.000362,0.000021,2.687649e-06,2.775179e-07,7.201764e-08,4.670266e-10,6.543773e-10,1.772712e-10
p1,0.014286,0.003473,0.000217,0.000025,9.333748e-07,1.037935e-07,0.000000e+00,4.868595e-09,1.404782e-09,2.759677e-11
p2,0.011583,0.003647,0.000274,0.000000,0.000000e+00,0.000000e+00,1.841681e-08,5.881519e-09,6.968631e-10,8.591398e-11
p3,0.025641,0.001462,0.000119,0.000007,8.372784e-07,9.504857e-08,9.099632e-09,3.415832e-09,3.267965e-10,4.876053e-11
p4,0.000000,0.001546,0.000246,0.000076,8.476176e-06,8.007130e-07,9.801318e-09,2.878273e-09,5.517026e-10,0.000000e+00
p5,0.013120,0.001937,0.000213,0.000000,0.000000e+00,0.000000e+00,4.271663e-08,4.428454e-09,0.000000e+00,9.049996e-11


In [15]:
probability_fwd(fwd, obs)

4.30042456759849e-10

#### 6). Алгоритм backward

In [16]:
def backward(hmm, obs):
    L = len(obs)
    bwd = pd.DataFrame(np.zeros((hmm.M,L)),index=hmm.transMatrix.index)
    for i in range(L-1,-1,-1):
        for s in hmm.obs_set:
            if i == L-1:
                b_sum = 1
            else:
                b_sum = 0
                for k_i,k in enumerate(hmm.obs_set):
                    b_sum += hmm.transMatrix.loc[s][k]*hmm.emissionMatrix.loc[k][obs[i]]*bwd[i+1][s]
            bwd[i][s] = b_sum
    return bwd

def probability_bwd(bwd, hmm, obs):
    prob = 0
    for s in hmm.obs_set:
        prob += bwd[0][s]*hmm.initialProb[s]*hmm.emissionMatrix.loc[s][obs[0]]
    return prob[0]

In [17]:
bwd = backward(hmm, obs)
print(bwd)

               0             1             2         3         4         5  \
p0  2.256196e-09  2.440179e-08  1.586943e-07  0.000001  0.000017  0.000209   
p1  4.090724e-09  4.322734e-08  3.362342e-07  0.000003  0.000027  0.000253   
p2  2.367663e-09  1.609846e-08  1.191044e-07  0.000001  0.000014  0.000184   
p3  3.502611e-09  2.448582e-08  2.197420e-07  0.000002  0.000021  0.000243   
p4  3.996069e-09  2.523553e-08  2.169017e-07  0.000002  0.000018  0.000175   
p5  2.859856e-09  2.653060e-08  1.821360e-07  0.000002  0.000018  0.000198   

           6         7         8    9  
p0  0.002536  0.029696  0.193127  1.0  
p1  0.002390  0.021829  0.169795  1.0  
p2  0.002412  0.021805  0.161323  1.0  
p3  0.002805  0.025516  0.228989  1.0  
p4  0.001679  0.012622  0.108489  1.0  
p5  0.002153  0.022183  0.152287  1.0  


In [18]:
print(probability_bwd(bwd, hmm, obs))

2.771828592668726e-10


#### 7). Постериорные вероятности

In [22]:
def prob_posterior(hmm, obs):
    fwd = forward(hmm, obs)
    bwd = backward(hmm, obs)
    pr = probability_fwd(fwd, obs)
    posterior = fwd.mul(bwd).div(pr)
    return posterior.div(posterior.sum())

In [23]:
prob_posterior(hmm, obs)

,0,1,2,3,4,5,6,7,8,9
p0,0.230854,0.032886,0.204178,0.115227,0.189969,0.234268,0.505855,0.029501,0.206497,0.412218
p1,0.210832,0.433350,0.258873,0.276206,0.102270,0.105900,0.000000,0.226069,0.389741,0.064172
p2,0.098941,0.169475,0.115895,0.000000,0.000000,0.000000,0.123044,0.272794,0.183689,0.199780
p3,0.324012,0.103318,0.093230,0.051719,0.072396,0.093343,0.070714,0.185399,0.122274,0.113385
p4,0.000000,0.112600,0.189994,0.556848,0.635365,0.566490,0.045598,0.077279,0.097798,0.000000
p5,0.135362,0.148370,0.137831,0.000000,0.000000,0.000000,0.254788,0.208958,0.000000,0.210444
